# Project: Research No-Show appointments

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

For this project I will be analyzing datasets from the year 2016 containing 100k medical appointments in Brazil attempting to determine which factors help predict if a customer will show up for their scheduled appointment.

this dataset contains individual appointment records containing specific customer information such as age, neighborhood, scholarship (Brazilian welfare program), and potential ailments.  The records also track whether or not the customer received an SMS message prior to their appoinment.  

After determining a baseline No-Show rate for all appointments in 2016, I will be focusing on the following questions to determine if i can better understand what factors have No-Show rates higher or lower than the baseline:

<ul>
<li><a href="#Q1">What is the current No-Show rate for appointments for all of 2016?</a></li>
<li><a href="#Q2">Are the No-Show rates higher with males or females?</a></li>
<li><a href="#Q3">Does receiving an SMS message prior to appointment affect the No-Show rate?</a></li>
<li><a href="#Q4">Do scholarship recipients have a higher or lower No-Show rate than the baseline?</a></li>
<li><a href="#Q5">How does the No-Show rate differ by month of the appointment?</a></li>
<li><a href="#Q6">How do the No-Show rates differ between customers with specific ailments?</a></li>
<li><a href="#Q7">Which neighborhood produces the highest counts of No-Shows?</a></li>
<li><a href="#Q8">What are the no-show rates for specific age ranges?</a></li>
<li><a href="#Q9">How is the No-Show rate affected by the time between the scheduling of the appointment and the actual appointment?</a></li>
</ul>



In [17]:
#importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Style and size of graphs
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(10,8)})

<a id='wrangling'></a>
## Data Wrangling

> In this section of the report, I will be loading the data, checking for cleanliness, and then trimming and cleaning the dataset for analysis.



In [18]:
# Load data and determine size of the dataframe

df = pd.read_csv('noshow_appts.csv')
print("The noshow Dataset contains (Rows,Columns) : ",df.shape)

The noshow Dataset contains (Rows,Columns) :  (110527, 14)


In [19]:
# Check for columns with missing data

df.isnull().value_counts()

PatientId  AppointmentID  Gender  ScheduledDay  AppointmentDay  Age    Neighbourhood  Scholarship  Hipertension  Diabetes  Alcoholism  Handcap  SMS_received  No-show
False      False          False   False         False           False  False          False        False         False     False       False    False         False      110527
dtype: int64

In [20]:
# View table sample

df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589980e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262960e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679510e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841190e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [21]:
# remove the dash from the no-show column and replace with an underscore.  check for fix
df.rename(columns=lambda x: x.strip().lower().replace("-", "_"), inplace=True)
df.head(1)

,patientid,appointmentid,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No


In [23]:
# correct spelling errors in column headers

df.rename(columns={'neighbourhood':'neighborhood', 'hipertension':'hypertension', 'handcap':'handicap','scheduledday':'sch_day', 'appointmentday':'app_day', }, inplace=True)
df.head(0)

,patientid,appointmentid,gender,sch_day,app_day,age,neighborhood,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,no_show


In [7]:
# Check for duplicate rows
dup_row = sum(df.duplicated())
print("There are", dup_row, "duplicated rows")

There are 0 duplicated rows


In [8]:
# Check for duplicate appointment numbers in AppointmentID column

dup_app = sum(df.appointmentid.duplicated())
print("there are", dup_app, "duplicate rows with matching appointment ids")

there are 0 duplicate rows with matching appointment ids


In [9]:
# delete PatientID & AppointmentID columns - they are not needed for this analysis

df.drop(['patientid','appointmentid'], axis=1, inplace=True)

# check to insure columns were dropped
df.head(1)

,gender,scheduledday,appointmentday,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received,no_show
0,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No


In [10]:
# check columns for value types

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   gender          110527 non-null  object
 1   scheduledday    110527 non-null  object
 2   appointmentday  110527 non-null  object
 3   age             110527 non-null  int64 
 4   neighbourhood   110527 non-null  object
 5   scholarship     110527 non-null  int64 
 6   hipertension    110527 non-null  int64 
 7   diabetes        110527 non-null  int64 
 8   alcoholism      110527 non-null  int64 
 9   handcap         110527 non-null  int64 
 10  sms_received    110527 non-null  int64 
 11  no_show         110527 non-null  object
dtypes: int64(7), object(5)
memory usage: 10.1+ MB


In [11]:
# check table stats

df.describe()

,age,scholarship,hipertension,diabetes,alcoholism,handcap,sms_received
count,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [12]:
# Total Appointments

tot_apt = df['no_show'].count()
print("The total number of appointments is:", tot_apt) 


The total number of appointments is: 110527


In [14]:
# Total Customers to showed up for appointment

df['no_show'].value_counts()

No     88208
Yes    22319
Name: no_show, dtype: int64

In [16]:
# Variables for show and no_show appointments
ns_y = df[df.no_show == 'Yes'].shape[0]
ns_y

22319

In [79]:
# Variables to store sum values for scholarships and disorders

sch = df.scholarship.sum()
hip = df.hipertension.sum()
dia = df.diabetes.sum()
alc = df.alcoholism.sum()
han = df.handcap.sum()
sms = df.sms_received.sum()

In [27]:
## Data Cleaning Conclusion 

>In reviewing the data, the following was determined
<ul>
<li>The first two columns, PatientId and AppointmentID were not relevent to the analysis</LI>
<li>The final column "No-show" had a dash instead of an underscore</LI>
<li>There were no columns with missing data</LI>
<li>There were no rows with duplicate information</LI>
<li>There were no rows with duplicated appointmentIds</LI>
</UL>

Steps taken to clean the data...
Dropped the PatientId and AppointmentId columns

SyntaxError: invalid syntax (<ipython-input-27-6e66121025d2>, line 3)

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!